<a href="https://colab.research.google.com/github/coding-geek21/BPN_and_FCM_Soft_Computing/blob/master/BPN_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Back Propagation Network Implementation**

## **Objective:**
#### To implement back propagation network on Iris dataset to classify the Iris species

## **Steps Involved**


1.   Import Modules
2.   Load Dataset
3.   Data Preprocessing
4.   Initializing the network
5.   Training network
> *   Forward propagation
  *   Backward Propagation Error
  *   Weight Updation
6.   Forward propagation
> *   input activation function
  *   finding transfer i.e f(yin)
7.   Back propation of error
> *   finding error value (tk-yk)
  *   finding transfer derivative i.e f'(yin)
8.   Weight updation
9.   Predict function
10.  Store the accuracy for all the folds 
11.  Find the mean accuracy for the folds






#### **IMPORTING MODULES**

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from random import randrange
from random import random
from csv import reader
from math import exp

#### **LOADING DATASET**

In [43]:
df=pd.read_csv("Iris.csv")
df.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [44]:
# Loading CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset


#### **DATA PREPROCESSING**

In [45]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

In [46]:
# Min max scaling

def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

#### **K-FOLD CROSS VALIDATION**

In [47]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [48]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		print("Actual Output:", actual[i] , "Predicted:", predicted[i])
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [49]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

#### **ACTIVATION FUNCTION**

In [50]:
 
# Calculate neuron activation for an input
# net input calculation yink = w0k + summation(wjk*zj)

def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

In [51]:
# Transfer neuron activation  #binary sigmoidal function
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))  

### **FORWARD PROPAGATION**

In [52]:
# Forward propagate input to a network output
# Feed forward phase

def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)     # binary sigmoidal function
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

In [53]:
# Calculate the derivative of an neuron output   
# f'(yink)

def transfer_derivative(output):
	return output * (1.0 - output)

### **BACKPROPAGATING ERROR**

In [54]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]

				#  error term (tk-yk)
				
				errors.append(neuron['output'] - expected[j])   
		for j in range(len(layer)):
			neuron = layer[j]

			# del k = (tk-yk)*f'(yink)

			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])  

### **WEIGHT UPDATION**

In [55]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):

				# change in weights = learning rate * delta * input 
				# change in weights for w0 = learning rate * delta
				
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

### **TRAINING**

In [56]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

In [57]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

### **PREDICTION**

In [58]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

### **BPN**

In [59]:
 # Backpropagation Algorithm
 
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	print("Inputs:", n_inputs, "Hidden:", n_hidden, "Output:", n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [60]:
# load and prepare data
filename = 'Iris_dataset.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
 

# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)


# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

Inputs: 4 Hidden: 5 Output: 3
Actual Output: 0 Predicted: 0
Actual Output: 0 Predicted: 0
Actual Output: 0 Predicted: 0
Actual Output: 2 Predicted: 2
Actual Output: 2 Predicted: 2
Actual Output: 1 Predicted: 1
Actual Output: 0 Predicted: 0
Actual Output: 1 Predicted: 1
Actual Output: 1 Predicted: 1
Actual Output: 1 Predicted: 1
Actual Output: 2 Predicted: 2
Actual Output: 0 Predicted: 0
Actual Output: 2 Predicted: 2
Actual Output: 1 Predicted: 1
Actual Output: 1 Predicted: 1
Actual Output: 0 Predicted: 0
Actual Output: 1 Predicted: 1
Actual Output: 2 Predicted: 2
Actual Output: 0 Predicted: 0
Actual Output: 0 Predicted: 0
Actual Output: 1 Predicted: 1
Actual Output: 1 Predicted: 1
Actual Output: 2 Predicted: 2
Actual Output: 0 Predicted: 0
Actual Output: 2 Predicted: 2
Actual Output: 2 Predicted: 2
Actual Output: 2 Predicted: 0
Actual Output: 1 Predicted: 1
Actual Output: 1 Predicted: 1
Actual Output: 1 Predicted: 1
Inputs: 4 Hidden: 5 Output: 3
Actual Output: 2 Predicted: 2
Actual Out

### **Result**

In [61]:
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [96.66666666666667, 96.66666666666667, 100.0, 93.33333333333333, 96.66666666666667]
Mean Accuracy: 96.667%


## **Conclusion:**
#### The above python program to implement Back propagation network has been successfully completed and executed and classification of Iris species is done successfully